## Week 5 
## Find the income using Support Vector Machines!


From the link to adult.zip download the data set. First, take a look at the data. You can see that the data contains categorical data as well. First of run a random forests and measure your performance.


the first part fo the assignment, I am gonig to use part of Abtin's code to create the functions and classes needed to run Random Forest and measure the performance 


In [1]:
import copy
import math
import random
import numpy as np 
import random 
import matplotlib.pyplot as plt

from statistics import mean, stdev
from collections import defaultdict

In [2]:
def Entropy(prob_X):
    """
    This is the entropy function that computes 
    entropy  of a  given  random  variables  X 
    and with  their corresponding probabilities 
    p_i based on the definition in:
    
    Shanon and Weaver, 1949 
    
    -> Links to paper :
    --> http://math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf
    --> https://ieeexplore.ieee.org/document/6773024
    
    
              Entropy = Σ_i p_i * log2 (p_i)
              
              
    INPUT:
    -------
            prob_X (a list/array of variables):  
       
       it should contains all the  probabilities of 
       the underlying random variable, each element
       expected to be a (0 <= float) and should
       add up to 1. (Else will be normalized)
       
       
       
    OUTPUT:
    -------
            Entropy (float): Entropy bits 
    """
    import math
    _sum_ = 0
    
    _tot_ = 0
    # checks
    for prob in prob_X:
        assert prob >= 0, "Negative probability is not accepted!!!"
        _tot_ += prob
    
#     if _tot_!=1:
#         print("Inputs are not normalized added up to {}, will be normalized!!".format(_tot_))
    
    for prob in prob_X:
        if _tot_==0:
            continue
            
        prob = prob/_tot_
        if prob == 0:
            pass
        else:
            _sum_ += prob * math.log2(prob)
        
    return abs(_sum_)


def Boolean_Entropy(q):
    """
    Finds the entropy for a Boolean random variable. 
    
    INPUT:
    ------
           q (float) : is expected to be between 0 and 1 (else AssertionError)
           
    OUTPUT:
    -------
            Entropy (float) : Entropy of a throwing a coin with chances 
                              of P(H, T) = (q, 1 - q) in bits
                              
                              
    """
    assert q >= 0 and q <= 1, "q = {} is not between [0,1]!".format(q)
        
    return Entropy([q, 1-q])


def Boolean_Entropy_counts(p, n):
    """
    Finds the entropy for a Boolean random variable. 
    
    INPUT:
    ------
           p (int or float) : Number or relative fraction of positive instances
           n (int or float) : Number or relative fraction of negative instances
           
    OUTPUT:
    -------
            Entropy (float) : Entropy of a throwing a coin with chances 
                              of P(H, T) = (q, 1 - q) in bits
                              
                              with q = p / (n + p)
                              
    """
    if n==0 and p==0:
        return 0
    q = p / (n + p)
    return Boolean_Entropy(q)

In [3]:
def euclidean_distance(X, Y):
    return math.sqrt(sum((x - y)**2 for x, y in zip(X, Y)))


def cross_entropy_loss(X, Y):
    n=len(X)
    return (-1.0/n)*sum(x*math.log(y) + (1-x)*math.log(1-y) for x, y in zip(X, Y))


def rms_error(X, Y):
    return math.sqrt(ms_error(X, Y))


def ms_error(X, Y):
    return mean((x - y)**2 for x, y in zip(X, Y))


def mean_error(X, Y):
    return mean(abs(x - y) for x, y in zip(X, Y))


def manhattan_distance(X, Y):
    return sum(abs(x - y) for x, y in zip(X, Y))


def mean_boolean_error(X, Y):
    return mean(int(x != y) for x, y in zip(X, Y))


def hamming_distance(X, Y):
    return sum(x != y for x, y in zip(X, Y))


In [4]:
def _read_data_set(data_file, skiprows=0, separator=None):
    with open(data_file, "r") as f:
        file = f.read()
        lines = file.splitlines()
        lines = lines[skiprows:]
        
    data_ = [[] for _ in range(len(lines))]
    
    for i, line in enumerate(lines):
        splitted_line = line.split(separator)
        float_line = []
        for value in splitted_line:
            try:
                value = float(value)
            except ValueError:
                if value=="":
                    continue
                else:
                    pass
            float_line.append(value)
        if float_line:
            data_[i] = float_line 
    
    for line in data_:
        if not line:
            data_.remove(line)
    
    return data_

def unique(seq):
    """
    Remove any duplicate elements from any sequence,
    works on hashable elements such as int, float,
    string, and tuple.
    """
    return list(set(seq))


def remove_all(item, seq):
    """Return a copy of seq (or string) with all occurrences of item removed."""
    if isinstance(seq, str):
        return seq.replace(item, '')
    else:
        return [x for x in seq if x != item]
    
    
def weighted_sample_with_replacement(n, seq, weights):
    """Pick n samples from seq at random, with replacement, with the
    probability of each element in proportion to its corresponding
    weight."""
    sample = weighted_sampler(seq, weights)

    return [sample() for _ in range(n)]
    

def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    import bisect

    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)

    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]
    
    
def mode(data):
    import collections
    """Return the most common data item. If there are ties, return any one of them."""
    [(item, count)] = collections.Counter(data).most_common(1)
    return item

# argmin and argmax

identity = lambda x: x

argmin = min
argmax = max


def argmin_random_tie(seq, key=identity):
    """Return a minimum element of seq; break ties at random."""
    return argmin(shuffled(seq), key=key)


def argmax_random_tie(seq, key=identity):
    """Return an element with highest fn(seq[i]) score; break ties at random."""
    return argmax(shuffled(seq), key=key)


def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items

def check_equal(iterator):
    iterator = iter(iterator)
    try:
        first = next(iterator)
    except StopIteration:
        return True
    return all(first == rest for rest in iterator)

def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

In [5]:
class Data_Set:
    """
    Defining a _general_ data set class for machine learning. 
    
    These are the following fields:
    
    >> data = Data_set

    data.examples:
           
           list of examples. Each one is a list contains of attribute values.

    
    data.attributes:
    
           list of integers to index into an example, so example[attribute]
           gives a value.
    
    
    data.attribute_names:
             
           list of names for corresponding attributes.
    
    
    data.target_attribute:
    
           The target attribute for the learning algorithm.
           (Default = last attribute)
    
    
    data.input_attributes:
             
           The list of attributes without the target.
    
    
    
    data.values:
             
           It is a list of lists in which each sublist is the
           set of possible values for the corresponding attribute. 
           If initially None, it is computed from the known examples
           by self.setproblem. If not None, bad value raises ValueError.
           
           
    data.distance_measure:
                 
           A measure  of  distance  function which takes two examples
           and returns a nonnegative number. It should be a symmetric 
           function. 
           (Defaults = mean_boolean_error) : can handle any field types.
           
           
    data.file_info:
           
           This should be a tuple that contains:
        (file_address, number of rows to skip, separator)
           
           
           
    data.name:
            
           This is for naming the data set. 
    
    
    
    data.source:
    
            URL or explanation to the dataset main source
            
            
    data.excluded_attributes:
     
            List of attribute indexes to exclude from data.input_attributes.
            (indexes or names of the attributes)

    Normally, you call the constructor and you're done; then you just
    access fields like d.examples and d.target and d.inputs."""

    
    
    
    def __init__(self, examples=None, attributes=None,  attribute_names=None, 
                 target_attribute = -1,  input_attributes=None,  values=None, 
                 distance_measure = mean_boolean_error, name='',  source='', 
                 excluded_attributes=(), file_info=None):
        
        """
        Accepts any of DataSet's fields. Examples can also be a
        string or file from which to parse examples using parse_csv.
        Optional parameter: exclude, as documented in .setproblem().
        
        >>> DataSet(examples='1, 2, 3')
        <DataSet(): 1 examples, 3 attributes>
        """
        
        self.file_info = file_info
        self.name = name
        self.source = source
        self.values = values
        self.distance = distance_measure
        self.check_values_flag = bool(values)

        # Initialize examples from a list
        if examples is not None:
            self.examples = examples
        elif file_info is None:
            raise ValueError("No Examples! and No Address!")
        else:
            self.examples = _read_data_set(file_info[0], file_info[1], file_info[2])

        # Attributes are the index of examples. can be overwrite 
        if self.examples is not None and attributes is None:
            attributes = list(range(len(self.examples[0])))

        self.attributes = attributes
        
        # Initialize attribute_names from string, list, or to default
        if isinstance(attribute_names, str):
            self.attribute_names = attribute_names.split()
        else:
            self.attribute_names = attribute_names or attributes
            
        # set the definitions needed for the problem 
        self.set_problem(target_attribute, input_attributes=input_attributes, 
                         excluded_attributes=excluded_attributes)

        
        
    def get_attribute_num(self, attribute):
        if isinstance(attribute, str):
            return self.attribute_names.index(attribute)
        else:
            return attribute
        
    
    
    
    def set_problem(self, target_attribute, input_attributes=None, excluded_attributes=()):
        """
        By doing this we set the target, inputs and excluded attributes.
        
        This way, one DataSet can be used multiple ways. inputs, if specified,
        is a list of attributes, or specify exclude as a list of attributes
        to not use in inputs. Attributes can be -n .. n, or an attrname.
        Also computes the list of possible values, if that wasn't done yet."""
        
        self.target_attribute = self.get_attribute_num(target_attribute)
        
        exclude = [self.get_attribute_num(excluded) for excluded in excluded_attributes]
        
        if input_attributes:
            self.input_attributes = remove_all(self.target_attribute, input_attributes)
        else:
            inputs = []
            for a in self.attributes:
                if a != self.target_attribute and a not in exclude:
                    inputs.append(a)
            self.input_attributes = inputs

        if not self.values:
            self.update_values()
        self.sanity_check()
        
        
    def sanity_check(self):
        """Sanity check on the fields."""
        
        assert len(self.attribute_names) == len(self.attributes)
        assert self.target_attribute in self.attributes
        assert self.target_attribute not in self.input_attributes
        assert set(self.input_attributes).issubset(set(self.attributes))
        if self.check_values_flag:
            # only check if values are provided while initializing DataSet
            [self.check_example(example) for example in self.examples]
        

    def check_example(self, example):
        if self.values:
            for attr in self.attributes:
                if example[attr] not in self.values:
                    raise ValueError("Not recognized value of {} for attribute {} in {}"
                                     .format(example[attr], attr, example))
                    
                    
    def add_example(self, example):
        self.check_example(example)
        self.examples.append(example)
        
    
    def update_values(self):
        self.values = list(map(unique, zip(*self.examples)))
        
        
    def remove_examples(self, value=""):
        self.examples = [example for example in examples if value not in example]

    def sanitize(self, example):  
        """Copy of the examples with non input_attributes replaced by None"""
        _list_ = []
        for i, attr_i in enumerate(example):
            if i in self.input_attributes:
                _list_.append(attr_i)
            else:
                _list_.append(None)
        return _list_
    
    def train_test_split(self, test_fraction=0.3, Seed = 99):
        import numpy as np
        
        examples = self.examples
        atrrs = self.attributes
        atrrs_name = self.attribute_names
        target = self.target_attribute
        input_ = self.input_attributes
        name = self.name 

        np.random.seed(Seed)
        _test_index = np.random.choice(list(range(len(examples))), int(test_fraction * len(examples)), replace=False)

        test_examples = [example for i, example in enumerate(examples) if i in _test_index]

        train_examples = [example for example in examples if example not in test_examples]

        Test_data_set = Data_Set(examples=test_examples,
                                 attributes=atrrs,
                                 attribute_names=attr_names,
                                 target_attribute=target,
                                 input_attributes=input_,
                                 name=name + " Test set",)

        Train_data_set = Data_Set(examples=train_examples,
                                 attributes=atrrs,
                                 attribute_names=attr_names,
                                 target_attribute=target,
                                 input_attributes=input_,
                                 name=name + " Train set",)

        return Train_data_set, Test_data_set
                
        
    def __repr__(self):
        return '<DataSet({}): with {} examples, and {} attributes>'.format(
            self.name, len(self.examples), len(self.attributes))

        


In [6]:
class Decision_Branch:
    """
    A branch of a decision tree holds an attribute to test, and a dict
    of branches for each attribute's values.
    """

    def __init__(self, attribute, attribute_name=None, default_child=None, branches=None):
        """Initialize by specifying what attribute this node tests."""
        
        self.attribute = attribute
        self.attribute_name = attribute_name or attribute
        self.default_child = default_child
        self.branches = branches or {}

    def __call__(self, example):
        """Classify a given example using the attribute and the branches."""
        attribute_val = example[self.attribute]
        if attribute_val in self.branches:
            return self.branches[attribute_val](example)
        else:
            # return default class when attribute is unknown
            return self.default_child(example)

    def add(self, value, subtree):
        """Add a branch.  If self.attribute = value, move to the given subtree."""
        self.branches[value] = subtree

    def display_out(self, indent=0):
        name = self.attribute_name
        print("Test", name)
        for value, subtree in self.branches.items():
            print(" " * indent * 5, name, '=', value, "--->", end=" ")
            subtree.display_out(indent + 1)
        # New line
        print()

    def __repr__(self):
        return ('Decision_Branch({}, {}, {})'
                .format(self.attribute, self.attribute_name, self.branches))

In [7]:
class Decision_Leaf:
    """A simple leaf class for a decision tree that hold a result."""

    def __init__(self, result):
        self.result = result

    def __call__(self, example):
        return self.result

    def display_out(self, indent=0):
        print('RESULT =', self.result)

    def __repr__(self):
        return repr(self.result)

In [8]:
def Decision_Tree_Learner(dataset):
    """
    Learning Algorithm for a Decision Tree
    """

    target, values = dataset.target_attribute, dataset.values

    def decision_tree_learning(examples, attrs, parent_examples=()):
        if not examples:
            return plurality(parent_examples)
        elif same_class_for_all(examples):
            return Decision_Leaf(examples[0][target])
        elif not attrs:
            return plurality(examples)
        else:
            A = choose_important_attribute(attrs, examples)
            tree = Decision_Branch(A, dataset.attribute_names[A], plurality(examples))
            
            for (vk, exs) in split_by(A, examples):
                subtree = decision_tree_learning(
                    exs, remove_all(A, attrs), examples)
                tree.add(vk, subtree)
            return tree

    def plurality(examples):
        """Return the most occured target value for this set of examples.
        (If binary target this is the majority, otherwise plurality)"""
        most_occured = argmax_random_tie(values[target],
                                    key=lambda v: count_example_same_attr(target, v, examples))
        return Decision_Leaf(most_occured)

    def count_example_same_attr(attr, val, examples):
        """Count the number of examples that have example[attr] = val."""
        return sum(e[attr] == val for e in examples)

    def same_class_for_all(examples):
        """Are all these examples in the same target class?"""
        _class_ = examples[0][target]
        return all(example[target] == _class_ for example in examples)

    def choose_important_attribute(attrs, examples):
        """Choose the attribute with the highest information gain."""
        return argmax_random_tie(attrs,
                                 key=lambda a: information_gain(a, examples))

    def information_gain(attr, examples):
        """Return the expected reduction in entropy from splitting by attr."""
        def _entropy_(examples):
            count = []
            for val in values[target]:
                count.append(count_example_same_attr(target, val, examples))
            return Entropy(count)
        
        N = len(examples)
        remainder = sum((len(examples_i)/N) * _entropy_(examples_i)
                        for (v, examples_i) in split_by(attr, examples))
        return _entropy_(examples) - remainder

    def split_by(attr, examples):
        """Return a list of (val, examples) pairs for each val of attr."""
        return [(v, [e for e in examples if e[attr] == v])
                for v in values[attr]]

    return decision_tree_learning(dataset.examples, dataset.input_attributes)

In [9]:
def Random_Forest(dataset, n=5, verbose=False):
    """
    An ensemble of Decision Trees trained using bagging and feature bagging.
    
    bagging: Bootstrap aggregating
    """

    def data_bagging(dataset, m=0):
        """Sample m examples with replacement"""
        n = len(dataset.examples)
        return weighted_sample_with_replacement(m or n, dataset.examples, [1]*n)

    def feature_bagging(dataset, p=0.7):
        """Feature bagging with probability p to retain an attribute"""
        inputs = [i for i in dataset.input_attributes if probability(p)]
        return inputs or dataset.input_attributes

    def predict(example):
        if verbose:
            print([predictor(example) for predictor in predictors])
        return mode(predictor(example) for predictor in predictors)

    predictors = [Decision_Tree_Learner(Data_Set(examples=data_bagging(dataset),
                                                 attributes=dataset.attributes,
                                                 attribute_names=dataset.attribute_names,
                                                 target_attribute=dataset.target_attribute,
                                                 input_attributes=feature_bagging(dataset))) for _ in range(n)]

    return predict


#### Here, we are going to import the data file

In [10]:
adult_data = "/Users/User/OneDrive/FoundationsML/Week5/adult/adult.data" #home
#adult_data = "/Users/scabellos/OneDrive - Urban Science/User Documents/MastersDegree/ML/Week5/adult.data" #office

In [11]:
attr_names = ["age", "workclass", "fnlwgt", "education", "education-num", "maritalstatus", "occupation", "relationship", "race", "sex","capitalgain","capitalloss","hoursperweek","nativecountry","IncomeYear"]

In [12]:
AD_dataset = Data_Set(name = "Adult dataset", 
                      target_attribute=14,
                      attribute_names=attr_names,
                      file_info=(adult_data, None, ","))

In [14]:
AD_dataset

<DataSet(Adult dataset): with 32561 examples, and 15 attributes>

#### Next, we are going to  : 


1. Split the data
2. Run Random Forest 
3. Measure the accuracy 

In [39]:
AD_Train, AD_test = AD_dataset.train_test_split(test_fraction=0.2, Seed=90)

In [40]:
RF_AD = Random_Forest(AD_Train, n = 10)

In [41]:
def Measure_accuracy(true_values, predictions):
    _sum_ = 0
    for truth, prediction in zip(true_values, predictions):
        if truth==prediction:
            _sum_+=1
    return _sum_/len(predictions)

In [42]:
rf_pridictions = [RF_AD(a[:-1]) for a in AD_test.examples]
true_Test_values = [example[AD_test.target_attribute] for example in AD_test.examples]

Measure_accuracy(true_Test_values, rf_pridictions)

0.7811732186732187

#### Accuracy on Random Forest is 78.11 %


## Transform Categorical data into numbers!



Now that we have more complicated algorithm, let’s make use out of them. But ﬁrst you should change your categorical data into real valued number which are needed for the SVM algorithm. Come up with a method that can do this translation



First, I am gonig to convert the dataset into a pandas dataframe 

In [13]:
AD_dataset.attribute_names

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'maritalstatus',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capitalgain',
 'capitalloss',
 'hoursperweek',
 'nativecountry',
 'IncomeYear']

In [14]:
import pandas as pd
    #Convert list of tuples to dataframe and set column names and indexes
AD_pd = pd.DataFrame(data=AD_dataset.examples,columns=AD_dataset.attribute_names,index=list(range(0,32561))) 

In [15]:
AD_pd.shape

(32561, 15)

Exploring the data, we are going to look for duplicates, missing values ... etc. 

In [16]:
AD_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age              32561 non-null float64
workclass        32561 non-null object
fnlwgt           32561 non-null float64
education        32561 non-null object
education-num    32561 non-null float64
maritalstatus    32561 non-null object
occupation       32561 non-null object
relationship     32561 non-null object
race             32561 non-null object
sex              32561 non-null object
capitalgain      32561 non-null float64
capitalloss      32561 non-null float64
hoursperweek     32561 non-null float64
nativecountry    32561 non-null object
IncomeYear       32561 non-null object
dtypes: float64(6), object(9)
memory usage: 4.0+ MB


In [17]:
# checking for duplicates 
AD_pd=AD_pd.drop_duplicates()

In [18]:
## Drop all "?"= missing values in the data 

AD_pd=AD_pd.drop(AD_pd[AD_pd['workclass']==' ?'].index)
AD_pd=AD_pd.drop(AD_pd[AD_pd['occupation']==' ?'].index)
AD_pd=AD_pd.drop(AD_pd[AD_pd['nativecountry']==' ?'].index)

In [19]:
AD_pd[AD_pd.isnull().any(axis=1)]

,age,workclass,fnlwgt,education,education-num,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,nativecountry,IncomeYear


In [20]:
AD_pd.shape

(30139, 15)

How many unique values we have in the data set ?

In [21]:
AD_pd.nunique()

age                 72
workclass            7
fnlwgt           20263
education           16
education-num       16
maritalstatus        7
occupation          14
relationship         6
race                 5
sex                  2
capitalgain        118
capitalloss         90
hoursperweek        94
nativecountry       41
IncomeYear           2
dtype: int64


### All these are categorical values - before we run SVM, we need to transform them into numbers. Lets analyze the data first to see if we can do some groupings and then use hot encoder to convert all categorical attributes 

#### workclass  --  categorical  ( 7 distinct values) 
#### education   --    categorical  ( 16 distinct values) 
#### maritalstatus --    categorical ( 7 distinct values) 
#### occupation   --    categorical ( 14 distinct values) 
#### relationship  --   categorical ( 6 distinct values) 
#### race    ---         categorical ( 5 distinct values) 
#### sex     ---        categorical ( 2 values )
#### nativecountry ---   categorical  ( 41 distinct values)  
#### IncomeYear  ---     categorical ( 2 values  )


In [22]:
#making a copy just in case i need to test things 
AD_pd_copy=AD_pd.copy()

### For IncomeYear (our target) we will use the cat encoder - 

#### I will categorize into 2 values. '>50K' = 1 and '<=50K' = 0 

In [24]:
AD_pd["IncomeYear"].value_counts()

 <=50K    22633
 >50K      7506
Name: IncomeYear, dtype: int64

In [25]:
AD_pd["IncomeYear_cat"] = AD_pd["IncomeYear"].astype('category')
AD_pd["IncomeYear_cat"].dtypes

CategoricalDtype(categories=[' <=50K', ' >50K'], ordered=False)

In [26]:
AD_pd["IncomeYear_cat"] = AD_pd["IncomeYear_cat"].cat.codes
AD_pd.head()

,age,workclass,fnlwgt,education,education-num,maritalstatus,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,nativecountry,IncomeYear,IncomeYear_cat
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K,0
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K,0
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K,0
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K,0
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K,0


In [27]:
AD_pd["IncomeYear_cat"].value_counts()

0    22633
1     7506
Name: IncomeYear_cat, dtype: int64

### 1st attribute I need to look into -  Sex -  


In [28]:
AD_pd["sex"].value_counts()

 Male      20366
 Female     9773
Name: sex, dtype: int64

### Now, we are going to analyze "Race" 

In [29]:
AD_pd["race"].value_counts()

 White                 25912
 Black                  2816
 Asian-Pac-Islander      894
 Amer-Indian-Eskimo      286
 Other                   231
Name: race, dtype: int64

Lets have a look at the 2 last categories 

In [30]:

race_analysis = AD_pd[AD_pd['race'].str.contains('Amer-Indian-Eskimo')& AD_pd['IncomeYear'].str.contains('>50K')]
race_analysis.shape

(34, 16)

In [31]:
race_analysis = AD_pd[AD_pd['race'].str.contains('Other')& AD_pd['IncomeYear'].str.contains('>50K')]
race_analysis.shape

(21, 16)

Only 55 out of the 7506 adults who make more than $50k  are of race Amer-Indian-Eskimo or Other. 
I am going to merge these 2 categories to make 1 and reduce the number of categories i need to encode. 


I'll group them all into other and review for duplicates again 

In [35]:
AD_pd["race"]= AD_pd["race"].replace(' Amer-Indian-Eskimo', ' Other') 

In [36]:
AD_pd["race"].value_counts()

 White                 25912
 Black                  2816
 Asian-Pac-Islander      894
 Other                   517
Name: race, dtype: int64

In [37]:
# checking for duplicates 
AD_pd=AD_pd.drop_duplicates()

### Marital Status now 

In [38]:
AD_pd["maritalstatus"].value_counts()

 Married-civ-spouse       14059
 Never-married             9711
 Divorced                  4212
 Separated                  939
 Widowed                    827
 Married-spouse-absent      370
 Married-AF-spouse           21
Name: maritalstatus, dtype: int64

#### In the context of providing Income to a household, I am gonig to group those individuals who do not have a spouse as 1Member_Family and the remaining as 2Member_Family

In [39]:
AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Never-married', '1M_Family')
AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Divorced', '1M_Family')
AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Separated', '1M_Family')
AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Married-spouse-absent', '1M_Family')
AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Widowed', '1M_Family')


AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Married-civ-spouse', '2M_Family')
AD_pd["maritalstatus"]= AD_pd["maritalstatus"].replace(' Married-AF-spouse', '2M_Family')



In [40]:
AD_pd["maritalstatus"].value_counts()

1M_Family    16059
2M_Family    14080
Name: maritalstatus, dtype: int64

###  Lets analyze Relationship

In [41]:
AD_pd["relationship"].value_counts()

 Husband           12457
 Not-in-family      7714
 Own-child          4462
 Unmarried          3211
 Wife               1406
 Other-relative      889
Name: relationship, dtype: int64

###  Occupation

In [42]:
AD_pd["occupation"].value_counts()

 Prof-specialty       4034
 Craft-repair         4025
 Exec-managerial      3991
 Adm-clerical         3719
 Sales                3584
 Other-service        3209
 Machine-op-inspct    1964
 Transport-moving     1572
 Handlers-cleaners    1349
 Farming-fishing       987
 Tech-support          911
 Protective-serv       644
 Priv-house-serv       141
 Armed-Forces            9
Name: occupation, dtype: int64

### Working Class 

In [43]:
AD_pd["workclass"].value_counts()

 Private             22264
 Self-emp-not-inc     2498
 Local-gov            2067
 State-gov            1279
 Self-emp-inc         1074
 Federal-gov           943
 Without-pay            14
Name: workclass, dtype: int64

In [44]:
workclass_analysis = AD_pd[AD_pd['workclass'].str.contains('Without-pay')& AD_pd['IncomeYear'].str.contains('>50K')]
workclass_analysis.shape

(0, 16)

#### Lets Remove those rows that dont report PAY as they dont add new information to the final target 

In [45]:
#AD_pd_copy=AD_pd_copy.drop(['workclass'==' Without-pay'], axis=1)

indexNames = AD_pd[AD_pd['workclass'].str.contains('Without-pay')].index
AD_pd.drop(indexNames , inplace=True)

In [46]:
AD_pd["workclass"].value_counts()

 Private             22264
 Self-emp-not-inc     2498
 Local-gov            2067
 State-gov            1279
 Self-emp-inc         1074
 Federal-gov           943
Name: workclass, dtype: int64

#### Now i am gonig ot create 3 bucket before onehotcoding -
#### Gov Employees / Private Sector / Self Employee 

In [47]:
AD_pd["workclass"]= AD_pd["workclass"].replace(' Federal-gov', 'gov')
AD_pd["workclass"]= AD_pd["workclass"].replace(' State-gov', 'gov')
AD_pd["workclass"]= AD_pd["workclass"].replace(' Local-gov', 'gov')
AD_pd["workclass"]= AD_pd["workclass"].replace(' Self-emp-not-inc', 'self')
AD_pd["workclass"]= AD_pd["workclass"].replace(' Self-emp-inc', 'self')
AD_pd["workclass"]= AD_pd["workclass"].replace(' Private', 'priv')

In [48]:
AD_pd["workclass"].value_counts()

priv    22264
gov      4289
self     3572
Name: workclass, dtype: int64

#### Education 


In [49]:
AD_pd["education"].value_counts()

 HS-grad         9825
 Some-college    6666
 Bachelors       5042
 Masters         1626
 Assoc-voc       1307
 11th            1048
 Assoc-acdm      1007
 10th             820
 7th-8th          555
 Prof-school      542
 9th              455
 12th             377
 Doctorate        375
 5th-6th          287
 1st-4th          149
 Preschool         44
Name: education, dtype: int64

#### is equivalent to education number 

In [144]:
AD_pd["education-num"].value_counts()

9.0     9825
10.0    6666
13.0    5042
14.0    1626
11.0    1307
7.0     1048
12.0    1007
6.0      820
4.0      555
15.0     542
5.0      455
8.0      377
16.0     375
3.0      287
2.0      149
1.0       44
Name: education-num, dtype: int64

#### So we are not gonig to consider education ( categorical ) in our model 
we will drop it from the model 

#### Next, we look into Native Country 

In [51]:
AD_pd["nativecountry"].value_counts()                      

 United-States                 27474
 Mexico                          606
 Philippines                     187
 Germany                         128
 Puerto-Rico                     109
 Canada                          107
 India                           100
 El-Salvador                     100
 Cuba                             92
 England                          86
 Jamaica                          80
 South                            71
 China                            68
 Italy                            68
 Dominican-Republic               67
 Vietnam                          64
 Guatemala                        61
 Japan                            59
 Poland                           56
 Columbia                         56
 Iran                             42
 Taiwan                           42
 Haiti                            42
 Portugal                         34
 Nicaragua                        33
 Peru                             30
 Greece                           29
 

#### Lets group our native countries into continents. I will leave the USA alone since income from the USA is the most prominent class. 

#### I am also deleting the values for South since it does not represent a country 

In [52]:
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' United-States'  ,'  USA'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Mexico'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Philippines'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Germany'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Puerto-Rico'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Canada'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' El-Salvador'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' India'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Cuba'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' England'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Jamaica'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Italy'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' China'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Dominican-Republic'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Vietnam'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Guatemala'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Japan'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Poland'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Columbia'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Taiwan'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Haiti'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Iran'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Portugal'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Nicaragua'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Peru'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Greece'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' France'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Ecuador'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Ireland'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Hong'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Cambodia'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Trinadad&Tobago'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Thailand'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Laos'  ,'  Asia'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Yugoslavia'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Outlying-US(Guam-USVI-etc)'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Hungary'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Honduras'  ,'  Am'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Scotland'  ,'  Eur'  )
AD_pd["nativecountry"]=AD_pd["nativecountry"].replace(' Holand-Netherlands'  ,'  Eur'  )



AD_pd=AD_pd.drop(AD_pd[AD_pd['nativecountry']==' South'].index)

In [53]:
AD_pd["nativecountry"].value_counts()

  USA     27474
  Am       1454
  Asia      633
  Eur       493
Name: nativecountry, dtype: int64

In [54]:
AD_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30054 entries, 0 to 32560
Data columns (total 16 columns):
age               30054 non-null float64
workclass         30054 non-null object
fnlwgt            30054 non-null float64
education         30054 non-null object
education-num     30054 non-null float64
maritalstatus     30054 non-null object
occupation        30054 non-null object
relationship      30054 non-null object
race              30054 non-null object
sex               30054 non-null object
capitalgain       30054 non-null float64
capitalloss       30054 non-null float64
hoursperweek      30054 non-null float64
nativecountry     30054 non-null object
IncomeYear        30054 non-null object
IncomeYear_cat    30054 non-null int8
dtypes: float64(6), int8(1), object(9)
memory usage: 3.7+ MB


#### Lets work on the encoding  - I am going to use One Hot Encoding for 

#### Sex, Race ( after grouping ) , MaritalStatus ( after grouping ) , WorkClass , Occupation, Relationship, NativeCountry ( after grouping )

The strategy is to convert each category value into a new column and assigns a 1 or 0 (True/False) value to the column. This has the benefit of not weighting a value improperly but does have the downside of adding more columns to the data set.

We dont have so many features so it is going to work fine in our data set. Still I am going to keep analyzing the columns to identify other groups to help our module.

In [55]:

AD_pd=pd.get_dummies(AD_pd, columns=["sex"])
AD_pd=pd.get_dummies(AD_pd, columns=["race"])
AD_pd=pd.get_dummies(AD_pd, columns=["maritalstatus"])
AD_pd=pd.get_dummies(AD_pd, columns=["workclass"])
AD_pd=pd.get_dummies(AD_pd, columns=["occupation"])
AD_pd=pd.get_dummies(AD_pd, columns=["relationship"])
AD_pd=pd.get_dummies(AD_pd, columns=["nativecountry"])


In [56]:
AD_pd.head()

,age,fnlwgt,education,education-num,capitalgain,capitalloss,hoursperweek,IncomeYear,IncomeYear_cat,sex_ Female,...,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,nativecountry_ Am,nativecountry_ Asia,nativecountry_ Eur,nativecountry_ USA
0,39.0,77516.0,Bachelors,13.0,2174.0,0.0,40.0,<=50K,0,0,...,0,1,0,0,0,0,0,0,0,1
1,50.0,83311.0,Bachelors,13.0,0.0,0.0,13.0,<=50K,0,0,...,1,0,0,0,0,0,0,0,0,1
2,38.0,215646.0,HS-grad,9.0,0.0,0.0,40.0,<=50K,0,0,...,0,1,0,0,0,0,0,0,0,1
3,53.0,234721.0,11th,7.0,0.0,0.0,40.0,<=50K,0,0,...,1,0,0,0,0,0,0,0,0,1
4,28.0,338409.0,Bachelors,13.0,0.0,0.0,40.0,<=50K,0,1,...,0,0,0,0,0,1,1,0,0,0


#### Now, I am going to create a copy, drop the columns we are not using and implement our SVM model 

In [57]:
AD_pd_final=AD_pd.copy()

In [58]:
AD_pd_final=AD_pd_final.drop(columns=['education','IncomeYear'])


In [59]:
AD_pd_final.head()

,age,fnlwgt,education-num,capitalgain,capitalloss,hoursperweek,IncomeYear_cat,sex_ Female,sex_ Male,race_ Asian-Pac-Islander,...,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,nativecountry_ Am,nativecountry_ Asia,nativecountry_ Eur,nativecountry_ USA
0,39.0,77516.0,13.0,2174.0,0.0,40.0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
1,50.0,83311.0,13.0,0.0,0.0,13.0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2,38.0,215646.0,9.0,0.0,0.0,40.0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,1
3,53.0,234721.0,7.0,0.0,0.0,40.0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
4,28.0,338409.0,13.0,0.0,0.0,40.0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0


In [60]:
X=AD_pd_final.drop(columns=['IncomeYear_cat'])
y=AD_pd_final['IncomeYear_cat']

#### Split the data into train and test 

In [61]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=109) # 70% training and 30% test

#### Preprocess the that using the Standard Scaler. Standardize features by removing the mean and scaling to unit variance 


Standardization of a dataset is a common requirement for many machine learning estimators: they might behave badly if the individual features do not more or less look like standard normally distributed data (e.g. Gaussian with 0 mean and unit variance).

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

X_train_sc=scaler.fit_transform(X_train)
X_test_sc=scaler.fit_transform(X_test)


#### Now we are going to implement our SVM models , running the data through different kernels ( linear, polinomial, sigmoid .... ) and measure the accuracy for those 

In [72]:
from sklearn import metrics
from sklearn.svm import SVC

acc=[]
d={}
kernels = ["linear", "rbf", "poly", "sigmoid"] 

#for model in range(len(models)):
for i in kernels:
        clf=SVC(kernel=i)
        clf.fit(X_train_sc,y_train)
        y_pred=clf.predict(X_test_sc)
        acc.append(metrics.accuracy_score(y_test, y_pred))

d={'Kernel':kernels,'Accuracy':acc}
acc_frame=pd.DataFrame(d)
acc_frame



,Kernel,Accuracy
0,linear,0.845958
1,rbf,0.850394
2,poly,0.839969
3,sigmoid,0.781080


#### As we can see, rbf ( Radial Base function ) is the Kernel that provides the highest level of accuracy to Support Vector Machine algortihm.

#### Just to summarize, these are the steps i took to complete this assignment. 


#### 1. I imported all the data and run Random Forest and evaluated accuracy. 

#### 2. Next, I converted the data set into a dataframe and analyzed and grouped some of the categorical variables before using the OneHot encoding method to convert them into numbers. 

#### 3. I scaled the data appropiately using the Scaler package to standardize our features 

#### 4. And finally, I implemented a loop to run several SVM implementations and print the accuracy, as we can see, SVM Provides higher accuracy compared to Random Forest. 


##### Thank you, 
##### Silvia 

